In [175]:
import pandas as pd
import requests as r
from bs4 import BeautifulSoup
import re

In [176]:
url='https://www.marmiton.org/recettes/recette_tacos-mexicains_34389.aspx'

In [3]:
r.get(url)

<Response [200]>

In [7]:
soup=BeautifulSoup(r.get(url).content)

[]

In [ ]:
list_of selectors=['h1','div.recipe-author__author>span','div.recipe-infos-users__rate__stars>span','ul.mrtn-tags-list>li','span.recipe-infos-users__value','div.recipe-infos,div.recipe-infos__timmings__detail','div.recipe-infos__timmings__cooking,div.recipe-infos__timmings__preparation']

In [177]:
#Recipe Name
soup.select('h1')[0].text

'Tacos mexicains'

In [178]:
#Author name
soup.select('div.recipe-author__author>span')[0].text.strip('\n ')

'noor_16706694'

In [17]:
#recipe note
soup.select('div.recipe-infos-users__rate__stars>span')[0].text

'4.6/5'

In [27]:
#tags ('entrée, 'plat unique','plats mexicains)
[i.text for i in soup.select('ul.mrtn-tags-list>li')]

['Entrée', 'Plat unique', 'Plats mexicains']

In [30]:
#Nb likes
soup.select('span.recipe-infos-users__value')[1].text

'15k'

In [115]:
#Nb of portions
[i.replace('\n',' ') for i in soup.select('div.recipe-infos,div.recipe-infos__timmings__detail')[0].text.split('\n\n') if len(i)>1]

['temps 55 min', ' personnes 4', ' très facile', 'bon marché']

In [113]:
#cooking time  preparation,cuisonn
[re.sub('\t|\n|\xa0','',i.text) for i in soup.select('div.recipe-infos__timmings__cooking,div.recipe-infos__timmings__preparation')]


['Préparation: 30 min', 'Cuisson:25 min']

In [128]:
#Ingredients
[i.text.strip().replace('\n','') for i in soup.select('li.recipe-ingredients__list__item')]

['8 tortillas pour tacos (achetés, ou recette sur le site Marmiton)',
 '1 oignon blanc',
 '2 tomates',
 '1verre de coulis de tomate (10 cl)',
 '250g de boeuf haché',
 '1petite boîte de haricots rouges',
 '1/2 poivron vert',
 '8feuilles de laitue',
 'Cumin en poudre',
 'Poivre',
 'Sel',
 'Tabasco (facultatif)']

In [137]:
#Ustensiles
[re.sub('\n\n|\t*',' ',i.text) for i in soup.select('ul.recipe-utensils__list')]

['   \n     1   P o ê l e  5 4 , 0 7 € \n s u r   A m a z o n         1   p i n c e a u  9 , 4 9 € \n s u r   A m a z o n         1   c o u t e a u  T o p   3   d e s   c o u t e a u x \n s u r   M a r m i t o n         1   P o ê l e   à   c r ê p e s  1 7 , 9 9 € \n s u r   A m a z o n         1   d é n o y a u t e u r  1 5 , 9 9 € \n s u r   A m a z o n   ']

In [130]:
[i.text for i in soup.select('ul.recipe-utensils__list')]

['\n\n\n\n\n \n\n\t\t\t\t\t\t\t1 Poêle\n\n54,07€\nsur Amazon\n\n\n\n\n\n\n\n \n\n\t\t\t\t\t\t\t1 pinceau\n\n9,49€\nsur Amazon\n\n\n\n\n\n\n\n \n\n\t\t\t\t\t\t\t1 couteau\n\nTop 3 des couteaux\nsur Marmiton\n\n\n\n\n\n\n\n \n\n\t\t\t\t\t\t\t1 Poêle à crêpes\n\n17,99€\nsur Amazon\n\n\n\n\n\n\n\n \n\n\t\t\t\t\t\t\t1 dénoyauteur\n\n15,99€\nsur Amazon\n\n\n\n']

In [110]:
# recipe
re.sub('\t|\n','',soup.select('ol.recipe-preparation__list')[0].text)

"A la poêle, faire dorer l'oignon émincé dans un peu d'huile d'olive. Rajouter la viande, assaisonner et laisser cuire 5 min. Laver les feuilles de laitue. Couper les tomates et le poivron en petits dés. Incorporer le tout à la poêlée avec le coulis de tomate, et poursuivre la cuisson pendant 5 min. Egoutter les haricots rouges et les ajouter 2 min avant la fin de cuisson. Hors du feu, ajuster l'assaisonnement et saupoudrer généreusement de cumin; on peut aussi rajouter quelques gouttes de Tabasco. Garnir les tortillas de préparation et les refermer en les roulant comme des crêpes. Disposer 1 feuille de laitue sur chaque tacos avant de servir."

In [144]:
url1='https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=50&page=4'

In [145]:
soup1=BeautifulSoup(r.get(url1).content)

In [147]:
[i['href'] for i in soup1.select('a.recipe-card-link')]


['/recettes/recette_pate-a-crepes-des-plus-raffinees_49665.aspx',
 '/recettes/recette_gateau-tres-rapide-au-yaourt-et-au-chocolat_30510.aspx',
 '/recettes/recette_gratin-dauphinois-recette-originale_22307.aspx',
 '/recettes/recette_blinis-faciles-maison_22261.aspx',
 '/recettes/recette_fondant-au-chocolat-plus-que-fondant_55396.aspx',
 '/recettes/recette_genoise-ultra-rapide-et-pourtant-si-aerienne_49844.aspx',
 '/recettes/recette_gratin-dauphinois_13809.aspx',
 '/recettes/recette_chevre-en-feuillete_11921.aspx',
 '/recettes/recette_gaufres-legeres_22127.aspx',
 '/recettes/recette_cake-coeur-fondant-chocolat_371446.aspx',
 '/recettes/recette_riz-au-lait-de-ma-maman_20380.aspx',
 '/recettes/recette_brownie-vite-fait_12890.aspx',
 '/recettes/recette_panna-cotta_17991.aspx',
 '/recettes/recette_muffin-foret-noire_371306.aspx']

# Code aggregation

In [152]:
# 14 recipes per page with page from no figure then 2 to 833
#https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=10004&page=833

[]

In [203]:
#liste of links for the pages - start with 5pages to test before launching 833 pages...
# pb le site change de pattern de pagination 
def List_urls(NbPages):
    j=NbPages+1
    urls=[]
    for i in range(0,j):
        url=f'https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start={4+14*i}&page={i+1}'
        urls.append(url)
        i+=1
    return urls

page_urls=List_urls(2)    
print(page_urls)

['https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=4&page=1', 'https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=18&page=2', 'https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=32&page=3']


In [204]:
#For one given page look for the list of recipes (links)
def find_recipe_card_link(url_of_page):
    soup=BeautifulSoup(r.get(url_of_page).content)
    # list tof recipes url which are in the page
    recipes_of_page=[f'https://www.marmiton.org'+i['href'] for i in soup.select('a.recipe-card-link')]
    return recipes_of_page
        
find_recipe_card_link(page_urls)    

InvalidSchema: No connection adapters were found for '['https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=4&page=1', 'https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=18&page=2', 'https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=32&page=3']'

In [181]:
def Collect_data(recipes_of_page):
    recipe=dict() #dictionnary , 1 row of the final dataframe (Recipe name: [list of info])  recipe={'':[]}
    for url in recipes_of_page:
        list_of_infos=[]
        soup=BeautifulSoup(r.get(url).content) 
        #Recipe Name
        name=soup.select('h1')[0].text
        list_of_infos.append(soup.select('div.recipe-author__author>span')[0].text.strip('\n ')) # add author name
        list_of_infos.append(soup.select('div.recipe-infos-users__rate__stars>span')[0].text) # add note
        list_of_infos.append([i.text for i in soup.select('ul.mrtn-tags-list>li')])# add tags
        #add total cooking time, nb of people, Easy or not, cheap or not
        list_of_infos.extend([i.replace('\n',' ') for i in soup.select('div.recipe-infos,div.recipe-infos__timmings__detail')[0].text.split('\n\n') if len(i)>1])
        list_of_infos.append(soup.select('span.recipe-infos-users__value')[1].text) #nb likes
        #cookig and preparation times
        list_of_infos.extend([re.sub('\t|\n|\xa0','',i.text) for i in soup.select('div.recipe-infos__timmings__cooking,div.recipe-infos__timmings__preparation')])
        #ingredients
        list_of_infos.extend([i.text.strip().replace('\n','') for i in soup.select('li.recipe-ingredients__list__item')])
        #recipe (text)
        list_of_infos.extend(re.sub('\t|\n','',soup.select('ol.recipe-preparation__list')[0].text))
        recipe[name]=list_of_infos # add a row to the dictionnary . 1 row = Recipe name: [list of info]
       
    return 



IndexError: list index out of range

In [172]:
def create_dataFrame(recipe):
  pd.DataFrame(recipe)

In [ ]:
#'d4 = {}' 'for d in (d1, d2, d3): d4.update(d)'

In [ ]:
# Error Handling function

In [ ]:
# Launch function terminal

In [200]:
url_page='https://www.marmiton.org/recettes/recherche.aspx?type=all&aqt=&start=18&page=2'
soup=BeautifulSoup(r.get(url_page).content)
[f'https://www.marmiton.org'+i['href'] for i in soup.select('a.recipe-card-link')]

['https://www.marmiton.org/recettes/recette_chouquettes_30622.aspx',
 'https://www.marmiton.org/recettes/recette_gateau-au-chocolat-des-ecoliers_20654.aspx',
 'https://www.marmiton.org/recettes/recette_lasagnes-a-la-bolognaise_18215.aspx',
 'https://www.marmiton.org/recettes/recette_mousse-au-chocolat-facile_13585.aspx',
 'https://www.marmiton.org/recettes/recette_spritz-dolce_392358.aspx',
 'https://www.marmiton.org/recettes/recette_gateau-au-chocolat-fondant-rapide_166352.aspx',
 'https://www.marmiton.org/recettes/recette_flan-patissier-traditionnel_15330.aspx',
 'https://www.marmiton.org/recettes/recette_canneles-bordelais_11439.aspx',
 'https://www.marmiton.org/recettes/recette_tarte-chocolat-poires-facile_14023.aspx',
 'https://www.marmiton.org/recettes/recette_choux-feuilletes-creme-au-chocolat-au-lait-et-smarties_371458.aspx',
 'https://www.marmiton.org/recettes/recette_pain-d-epices_11087.aspx',
 'https://www.marmiton.org/recettes/recette_pate-a-crepes-simple_27121.aspx',
 'htt

In [ ]:
https://www.marmiton.org/recettes/recette_chouquettes_30622.aspx